In [0]:
%sql

WITH silver_base AS (
  SELECT
    -- Business keys
    AccountName              AS account_name,
    ID                       AS owned_champion_id,

    -- Champion identity (not unique to owned copy)
    HeroID                   AS champion_id,

    -- Descriptors
    Name                     AS champion_name,
    Rank                     AS rank,
    Level                    AS level,
    EmpowerLevel             AS empower_level,
    Rarity                   AS rarity,
    Affinity                 AS affinity,
    Faction                  AS faction,

    -- Scroll usage
    UsedT1MasScrolls         AS used_t1_mastery_scrolls,
    UnUsedT1MasScrolls       AS unused_t1_mastery_scrolls,
    UsedT2MasScrolls         AS used_t2_mastery_scrolls,
    UnUsedT2MasScrolls       AS unused_t2_mastery_scrolls,
    UsedT3MasScrolls         AS used_t3_mastery_scrolls,
    UnUsedT3MasScrolls       AS unused_t3_mastery_scrolls,

    -- Stats
    HP                       AS hp,
    ATK                      AS atk,
    DEF                      AS def,
    CritRate                 AS crit_rate,
    CritDamage               AS crit_damage,
    SPD                      AS spd,
    ACC                      AS acc,
    RES                      AS res,

    -- Blessing
    BlessingID               AS blessing_id,
    BlessingGrade            AS blessing_grade,

    -- Lineage / audit (keep)
    run_id                   AS run_id,
    source_file              AS source_file,
    snapshot_ts              AS snapshot_ts,
    snapshot_date            AS snapshot_date,
    snapshot_version         AS snapshot_version,
    schema_version           AS schema_version

FROM workspace.raid.bronze_champindex
),
files AS (
    SELECT DISTINCT
        account_name,
        source_file,
        snapshot_date,
        snapshot_version
    FROM silver_base
),

latest_file AS (
    SELECT
        account_name,
        source_file,
        ROW_NUMBER() OVER (
            PARTITION BY account_name
            ORDER BY snapshot_date DESC, snapshot_version DESC
        ) AS rn
    FROM files
)

SELECT
    s.account_name,
    s.owned_champion_id,
    s.champion_id,
    s.champion_name,
    s.rank,
    s.level,
    s.empower_level,
    s.rarity,
    s.affinity,
    s.faction,

    s.used_t1_mastery_scrolls,
    s.unused_t1_mastery_scrolls,
    s.used_t2_mastery_scrolls,
    s.unused_t2_mastery_scrolls,
    s.used_t3_mastery_scrolls,
    s.unused_t3_mastery_scrolls,

    s.hp,
    s.atk,
    s.def,
    s.crit_rate,
    s.crit_damage,
    s.spd,
    s.acc,
    s.res,

    s.blessing_id,
    s.blessing_grade,

    s.run_id,
    s.source_file,
    s.snapshot_ts,
    s.snapshot_version,
    s.snapshot_date,
    s.schema_version

FROM silver_base s
JOIN latest_file l
  ON s.source_file = l.source_file
WHERE l.rn = 1
;